# Machine Learning Engineer Nanodegree
## Lin Muqing
## Capstone Model Iteration

In [1]:
# Import libraries necessary for this project
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import gc
import seaborn as sns
from time import time
from IPython.display import display # Allows the use of display() for DataFrames
import os
import matplotlib.pyplot as plt
import collections

# Pretty display for notebooks
%matplotlib inline